# Metacognitive Prompting

LLMに人間の認知プロセスを模倣させることで精度を改善した手法．
- 論文:https://arxiv.org/abs/2308.05342

<a href="https://colab.research.google.com/github/fuyu-quant/data-science-wiki/blob/develop/nlp/llm_prompt_engineering/metacognitive_promting.ipynb" target="_blank" rel="noopener noreferrer"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from langchain.llms import OpenAI

llm_model = OpenAI(model_name= 'gpt-3.5-turbo', temperature=0)

/Users/tanakatouma/vscode/data-science-wiki/.venv/lib/python3.9/site-packages/langchain/llms/openai.py:202: UserWarning: You are trying to use a chat model. This way of initializing it is no longer supported. Instead, please use: `from langchain.chat_models import ChatOpenAI`
  warnings.warn(
/Users/tanakatouma/vscode/data-science-wiki/.venv/lib/python3.9/site-packages/langchain/llms/openai.py:790: UserWarning: You are trying to use a chat model. This way of initializing it is no longer supported. Instead, please use: `from langchain.chat_models import ChatOpenAI`
  warnings.warn(


### 通常のプロンプト(Few-shot prompting)

In [2]:
prompt = """
質問1: 「世界で最も美しいビーチは何ですか？」と質問2:「最も美しいビーチは何ですか？」、
これら2つの質問がお互いの言い換えであるかどうかを判断してください。
"""

In [3]:
output = llm_model(prompt)
print(output)

これらの2つの質問はお互いの言い換えではありません。質問1は「世界で最も美しいビーチは何ですか？」という具体的な範囲を指定しており、質問2は単に「最も美しいビーチは何ですか？」と一般的な美しいビーチを尋ねています。質問1は特定の範囲を指定しているため、回答もより具体的になる可能性があります。


### Metacognitive Prompting

In [4]:
prompt1 = """
質問1: 「世界で最も美しいビーチは何ですか？」と質問2:「最も美しいビーチは何ですか？」、
これら2つの質問がお互いの言い換えであるかどうかを判断してください。
Q:両方の質問に対するあなたの理解を明確にしてください。
A:
"""

In [5]:
output1 = llm_model(prompt1)
print(output1)

質問1: 「世界で最も美しいビーチは何ですか？」
この質問は、世界中のビーチの中で最も美しいビーチを尋ねています。

質問2: 「最も美しいビーチは何ですか？」
この質問も、最も美しいビーチを尋ねていますが、特定の地域や範囲については言及していません。

これらの質問は意味的には非常に似ていますが、質問1は「世界で」という限定語を使用しており、特定の地域や範囲について尋ねていることを示しています。一方、質問2は特定の地域や範囲については言及せず、単に最も美しいビーチを尋ねています。したがって、これらの質問はお互いの言い換えではありません。


In [6]:
prompt2 = """
質問1: 「世界で最も美しいビーチは何ですか？」と質問2:「最も美しいビーチは何ですか？」、
これら2つの質問がお互いの言い換えであるかどうかを判断してください。
Q:両方の質問に対するあなたの理解を明確にしてください。
A:{output1_}
Q:主題、文脈、意味内容に基づく類似性の予備的識別を行う。
A:
""".format(
    output1_=output1
    )

In [7]:
output2 = llm_model(prompt2)
print(output2)

主題: 両方の質問は「美しいビーチ」に関連しています。
文脈: 両方の質問はビーチに関する情報を尋ねています。
意味内容: 両方の質問は最も美しいビーチについて尋ねています。

以上の要素から、両方の質問は主題、文脈、意味内容の面で類似していると言えます。ただし、質問1は特定の地域や範囲について尋ねている点で質問2と異なります。


In [8]:
prompt3 = """
質問1: 「世界で最も美しいビーチは何ですか？」と質問2:「最も美しいビーチは何ですか？」、
これら2つの質問がお互いの言い換えであるかどうかを判断してください。
Q:両方の質問に対するあなたの理解を明確にしてください。
A:{output1_}
Q:主題、文脈、意味内容に基づく類似性の予備的識別を行う。
A:{output2_}
Q:予備的な分析を批判的に評価する。質問がパラフレーズであるという最初の評価に確信が持てない場合は、再評価を試みる。
A:
""".format(
    output1_=output1,
    output2_=output2
    )

In [9]:
output3 = llm_model(prompt3)
print(output3)

予備的な分析は適切に行われており、質問1と質問2の類似性と相違点を明確に示しています。質問1は特定の地域や範囲について尋ねている一方、質問2は特定の地域や範囲について言及していません。この違いにより、質問1と質問2はお互いの言い換えではないという結論が導かれます。

再評価を試みる必要はありません。


In [10]:
prompt4 = """
質問1: 「世界で最も美しいビーチは何ですか？」と質問2:「最も美しいビーチは何ですか？」、
これら2つの質問がお互いの言い換えであるかどうかを判断してください。
Q:両方の質問に対するあなたの理解を明確にしてください。
A:{output1_}
Q:主題、文脈、意味内容に基づく類似性の予備的識別を行う。
A:{output2_}
Q:予備的な分析を批判的に評価する。質問がパラフレーズであるという最初の評価に確信が持てない場合は、再評価を試みる。
A:{output3_}
Q:質問がパラフレーズであるかどうかの最終的な判断を確認し、判断の根拠を示します。
A:
""".format(
    output1_=output1,
    output2_=output2,
    output3_=output3
    )

In [11]:
output4 = llm_model(prompt4)
print(output4)

質問1と質問2は、意味的には非常に似ていますが、質問1は「世界で」という限定語を使用しており、特定の地域や範囲について尋ねていることを示しています。一方、質問2は特定の地域や範囲については言及せず、単に最も美しいビーチを尋ねています。したがって、これらの質問はお互いの言い換えではありません。この判断は、質問の主題、文脈、意味内容の面での類似性と相違点に基づいています。


In [12]:
prompt5 = """
質問1: 「世界で最も美しいビーチは何ですか？」と質問2:「最も美しいビーチは何ですか？」、
これら2つの質問がお互いの言い換えであるかどうかを判断してください。
Q:両方の質問に対するあなたの理解を明確にしてください。
A:{output1_}
Q:主題、文脈、意味内容に基づく類似性の予備的識別を行う。
A:{output2_}
Q:予備的な分析を批判的に評価する。質問がパラフレーズであるという最初の評価に確信が持てない場合は、再評価を試みる。
A:{output3_}
Q:質問がパラフレーズであるかどうかの最終的な判断を確認し、判断の根拠を示します。
A:{output4_}
""".format(
    output1_=output1,
    output2_=output2,
    output3_=output3,
    output4_=output4
    )

In [13]:
output5 = llm_model(prompt5)
print(output5)

質問1と質問2はお互いの言い換えではないと判断されます。質問1は特定の地域や範囲について尋ねている一方、質問2は特定の地域や範囲について言及していません。この判断は、質問の主題、文脈、意味内容の面での類似性と相違点に基づいています。


In [14]:
prompt6 = """
質問1: 「世界で最も美しいビーチは何ですか？」と質問2:「最も美しいビーチは何ですか？」、
これら2つの質問がお互いの言い換えであるかどうかを判断してください。
Q:両方の質問に対するあなたの理解を明確にしてください。
A:{output1_}
Q:主題、文脈、意味内容に基づく類似性の予備的識別を行う。
A:{output2_}
Q:予備的な分析を批判的に評価する。質問がパラフレーズであるという最初の評価に確信が持てない場合は、再評価を試みる。
A:{output3_}
Q:質問がパラフレーズであるかどうかの最終的な判断を確認し、判断の根拠を示します。
A:{output4_}
Q:自分の分析に対する確信度（0～100％）を評価し、その確信度の説明をする。
A:
""".format(
    output1_=output1,
    output2_=output2,
    output3_=output3,
    output4_=output4,
    output5_=output5
    )

In [15]:
output6 = llm_model(prompt6)
print(output6)

自分の分析に対する確信度は90％です。質問1と質問2の類似性と相違点を明確に示し、それぞれの質問が異なる範囲を尋ねていることを示しています。ただし、確信度が100％ではないのは、他の文脈や意味の解釈が考えられるためです。しかし、現時点では質問1と質問2はお互いの言い換えではないという結論に自信を持っています。
